# News Classifier with BERT

In [ ]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
  return tokenizer(
      examples["text"],
      padding='max_length',
      truncation=True,
      max_length = 128
  )

tokenized_datasets = dataset.map(tokenize_function, batched = True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_columns({"label":"labels"})
tokenized_datasets.set_format("torch")


# make pretrained BERT for classification

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=4)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# training setup

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay =0.01,
    logging_dir="./logs",
    load_best_model_at_end=True
)


# Evaluation Matrics (accuracy + f1)

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
import evaluate
import numpy as np

accuracy_matrics = evaluate.load("accuracy")
f1_matrics = evaluate.load("f1")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)

  accuracy = accuracy_matrics.compute(predictions=predictions, references=labels)
  f1 = f1_matrics.compute(predictions=predictions, references=labels , average="weighted")
  return {
      "accuracy": accuracy["accuracy"],
      "f1": f1["f1"]
  }

# trainer

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

/tmp/ipython-input-1077093389.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# Train the Model

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: businessofcoding (businessofcoding-subhanud-din) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.331500,0.315302,0.895658,0.895324
2,0.220300,0.293081,0.916447,0.916369


TrainOutput(global_step=15000, training_loss=0.3474284637451172, metrics={'train_runtime': 5527.793, 'train_samples_per_second': 43.417, 'train_steps_per_second': 2.714, 'total_flos': 1.578694680576e+16, 'train_loss': 0.3474284637451172, 'epoch': 2.0})

In [ ]:
result = trainer.evaluate()
print(result)

{'eval_loss': 0.29308074712753296, 'eval_accuracy': 0.9164473684210527, 'eval_f1': 0.9163689635502973, 'eval_runtime': 56.4608, 'eval_samples_per_second': 134.607, 'eval_steps_per_second': 8.413, 'epoch': 2.0}


# test on custom headline

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
text = "Apple unveils new AI-powered MacBooks at annual event"

inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

# Move every tensor inside inputs dict to GPU/CPU
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    outputs = model(**inputs)

predicted_class = outputs.logits.argmax(dim=-1).item()

labels = ["World", "Sports", "Business", "Sci/Tech"]
print("Predicted Topic:", labels[predicted_class])


Predicted Topic: Sci/Tech


In [ ]:
trainer.save_model("news-bert-model")
tokenizer.save_pretrained("news-bert-model")

('news-bert-model/tokenizer_config.json',
 'news-bert-model/special_tokens_map.json',
 'news-bert-model/vocab.txt',
 'news-bert-model/added_tokens.json')

In [1]:
!pip freeze > requirements.txt